# CIFAR-100 Dataset: Image Classification
## Nick Cantalupa and Sean Duffy
In this project, we will be training 4 seperate models useing different supervised machine learning techniques. These models will be classifying images using a dataset of 60,000 training images. 

## Loading dataset

In [1]:
import torch
from torchvision.datasets import CIFAR100, CIFAR10
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [15]:
ROOT_PATH = 'data'

BATCH_SIZE = 4

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# train_dataset = CIFAR100(root=ROOT_PATH, download=True, train=True, transform=transform)
# eval_dataset = CIFAR100(root=ROOT_PATH, train=False, transform=transform)

train_dataset = CIFAR10(root=ROOT_PATH, download=True, train=True, transform=transform)
eval_dataset = CIFAR10(root=ROOT_PATH, train=False, transform=transform)

train_data_loader = DataLoader(dataset=train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
eval_data_loader = DataLoader(dataset=eval_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified


In [ ]:
import json

with open('data/cifar-100-python/meta', 'rb') as file:
    meta = pickle.load(file)

with open("labels.json", 'w') as mf:
    json.dump(meta, mf)

In [ ]:
label_mapper = {}
for i , label in enumerate(meta["fine_label_names"]):
    label_mapper[i] = label
label_mapper

In [ ]:
for train_data, train_labels in train_data_loader:
    print(train_data.shape)
    print(train_labels.shape)
    break

for eval_data, eval_labels in eval_data_loader:
    print(eval_data.shape)
    print(eval_labels.shape)
    break

In [ ]:
i = 3792
img = train_data[i, :].permute(1,2,0)
plt.imshow(img)
plt.show()
print(label_mapper[int(train_labels[i])])

In [ ]:
train_data[0].shape

## Logistic Regression

In [ ]:
def validation_split(n, val_pct):
    n_val = int(n*val_pct)
    indexes = np.random.permutation(n)
    return indexes[:n_val], indexes[n_val:]

training_set, validation_set = validation_split(len(train_data), .80)
len(training_set)

In [ ]:
input_size = 3*32*32
num_classes = 100

#### Neural LR

In [ ]:
class LogisticRegressionTorch(torch.nn.Module):
    def __init__(self):
        super(LogisticRegressionTorch, self).__init__()
        self.linear = torch.nn.Linear(input_size, num_classes)

    def forward(self, xb):
        xb = xb.reshape(-1, input_size)
        output = self.linear(xb)
        return output

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.sum(preds == labels).item() / len(preds)

In [ ]:
def loss_batch(model, loss_fn, xb, yb, opt=None, metric=None):
    preds = model(xb)
    loss = loss_fn(preds, yb)
    if (opt is not None):
        loss.backward()
        opt.step()
        opt.zero_grad()
    metric_result = None
    if metric is not None:
        metric_result = metric(preds,yb)
    return loss.item(), len(xb), metric_result

In [ ]:
def evaluate(model, loss_fn, valid_dl, metric=None):
    with torch.no_grad():
        results = [loss_batch(model, loss_fn, xb, yb, metric=metric) for xb, yb, in valid_dl]
        losses, nums, metrics = zip(*results)
        total = np.sum(nums)
        total_loss = np.sum(np.multiply(losses, nums))
        avg_loss = total_loss/total
        avg_metric = None 
        if metric is not None:
            tot_metric = np.sum(np.multiply(metrics, nums))
            avg_metric = tot_metric/total
    return avg_loss, total, avg_metric

In [ ]:
def fit(epochs, model, loss_fn, opt, train_dl, valid_dl, metric=None):
    for epoch in range(epochs):
        for xb, yb in train_dl:
            loss, _, _ = loss_batch(model, loss_fn, xb, yb, opt)
        result = evaluate(model, loss_fn, valid_dl, metric)
        val_loss, total, val_metric = result

        if metric is None:
            print("Epoch[{}/{}], Loss: {:.4f}".format(epoch+1, epochs, val_loss))
        else:
            print("Epoch[{}/{}], Loss: {:.4f}, {}: {:.4f}".format(epoch+1, epochs, val_loss, metric.__name__, val_metric))

In [ ]:
learning_rate = 0.1
model = LogisticRegressionTorch()
optimizer =torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
fit(10, model, torch.nn.functional.cross_entropy, optimizer, train_data_loader, eval_data_loader, accuracy)

#### Flattened Logistic Regression

In [ ]:
flattened_data = []
for i in range(len(train_data)):
    if (i%1000 == 0):
        print(round((i/len(train_data))*100, 4),"%")
    flat = train_data[i].flatten()
    flat = flat.numpy()
    flattened_data.append(pd.Series(flat))

df = pd.DataFrame(flattened_data)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = df
y = train_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=400)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

## CNN

In [17]:
import torch.nn.functional as F

### CIFAR10

In [18]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 5
learning_rate = 0.001

# classes = list(label_mapper.keys())

In [20]:
train_data_loader

In [21]:
class ConvNN(torch.nn.Module):
    # def __init__(self):
    #     super(ConvNN, self).__init__()
    #     self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, padding=1)
    #     self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
    #     self.pool1 = torch.nn.MaxPool2d(2, 2)
    #     self.conv3 = torch.nn.Conv2d(64, 128, kernel_size=3, padding=1)
    #     self.pool2 = torch.nn.MaxPool2d(2, 2)
    #     self.conv4 = torch.nn.Conv2d(128, 256, kernel_size=3, padding=1)
    #     self.pool3 = torch.nn.MaxPool2d(2, 2)
    #     self.fc1 = torch.nn.Linear(4*4*256, 1024)
    #     self.fc2 = torch.nn.Linear(1024, 100)

    # def forward(self, x):
    #     x = F.relu(self.conv1(x))
    #     x = F.relu(self.conv2(x))
    #     x = self.pool1(x)
    #     x = F.relu(self.conv3(x))
    #     x = self.pool2(x)
    #     x = F.relu(self.conv4(x))
    #     x = self.pool3(x)
    #     x = x.view(-1, 4*4*256)
    #     x = F.relu(self.fc1(x))
    #     x = self.fc2(x)
    #     return x
    def __init__(self):
        super(ConvNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [22]:
model = ConvNN().to(device)

In [23]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)
n_steps = len(train_data_loader)

In [24]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_steps}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [2000/12500], Loss: 2.2821
Epoch [1/5], Step [4000/12500], Loss: 2.2701
Epoch [1/5], Step [6000/12500], Loss: 2.2733
Epoch [1/5], Step [8000/12500], Loss: 2.3898
Epoch [1/5], Step [10000/12500], Loss: 1.8459
Epoch [1/5], Step [12000/12500], Loss: 1.9091
Epoch [2/5], Step [2000/12500], Loss: 1.5968
Epoch [2/5], Step [4000/12500], Loss: 1.9115
Epoch [2/5], Step [6000/12500], Loss: 1.8843
Epoch [2/5], Step [8000/12500], Loss: 1.8545
Epoch [2/5], Step [10000/12500], Loss: 1.6123
Epoch [2/5], Step [12000/12500], Loss: 1.8100
Epoch [3/5], Step [2000/12500], Loss: 2.7025
Epoch [3/5], Step [4000/12500], Loss: 1.1148
Epoch [3/5], Step [6000/12500], Loss: 1.3757
Epoch [3/5], Step [8000/12500], Loss: 1.2874
Epoch [3/5], Step [10000/12500], Loss: 2.3755
Epoch [3/5], Step [12000/12500], Loss: 1.1509
Epoch [4/5], Step [2000/12500], Loss: 1.1642
Epoch [4/5], Step [4000/12500], Loss: 1.0308
Epoch [4/5], Step [6000/12500], Loss: 0.9176
Epoch [4/5], Step [8000/12500], Loss: 1.4165
Epoc

In [25]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in eval_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(BATCH_SIZE):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy of the network: 49.04 %
Accuracy of plane: 53.4 %
Accuracy of car: 80.5 %
Accuracy of bird: 31.7 %
Accuracy of cat: 41.5 %
Accuracy of deer: 28.5 %
Accuracy of dog: 35.3 %
Accuracy of frog: 58.7 %
Accuracy of horse: 65.4 %
Accuracy of ship: 58.7 %
Accuracy of truck: 36.7 %
